### importing libraries

In [50]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
stemmer = WordNetLemmatizer()
import pickle
from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import nltk
from transformers import TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('corpus')
nltk.download('vader_lexicon')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [26]:
!pip install transformers
    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### load data

In [27]:
train = pd.read_csv('twitter_training.csv',sep=',', names=['Tweet_ID','Entity','Sentiment','Tweet_content'])
#test = pd.read_csv('twitter_validation.csv',sep=',', names=['Tweet_ID','Entity','Sentiment','Tweet_content'])

### CLEANING

In [28]:
### drop duplicates
train.drop_duplicates(inplace=True)
### drop null values
train.dropna(inplace=True, axis=0)

In [29]:
stemmer = WordNetLemmatizer()

def clean_txt(x):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(x))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # word tokenize
    document = word_tokenize(document)
    # remove stop words
    document = [word for word in document if word not in stopwords.words('english')]
    # word lemmatization
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

In [30]:
#train[train['Sentiment'] == 'Irrelevant']

In [31]:
train['Tweet_content'] = train['Tweet_content'].apply(lambda x:clean_txt(x))

In [32]:
train = train[['Tweet_content' ,'Sentiment']]

### preprocessing

In [33]:
### labeling for target data
sentiment_dict = {'Positive':1, 
        'Neutral':0, 
        'Negative':2, 
        'Irrelevant':3}

train['Sentiment'] = train['Sentiment'].map(size_dict)

In [34]:
reviews = train['Tweet_content'].values.tolist()
labels = train['Sentiment'].tolist()

In [2]:
### splitting data
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [36]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [37]:
### encoding toknized data
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [38]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

### building a model

In [39]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=4)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [40]:
### set parameters and model complieing
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [24]:
### fitting data
model.fit(train_dataset.shuffle(100).batch(64),
          epochs=10,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(64))

Epoch 1/10


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


63/63 [==============================] - 98s 1s/step - loss: -2.9206 - accuracy: 0.2380 - val_loss: -3.6446 - val_accuracy: 0.2460
Epoch 2/10
63/63 [==============================] - 73s 1s/step - loss: -4.5900 - accuracy: 0.2693 - val_loss: -3.6446 - val_accuracy: 0.2490
Epoch 3/10
63/63 [==============================] - 73s 1s/step - loss: -4.5900 - accuracy: 0.2763 - val_loss: -3.6446 - val_accuracy: 0.2490
Epoch 4/10
63/63 [==============================] - 73s 1s/step - loss: -4.5892 - accuracy: 0.2612 - val_loss: -3.6446 - val_accuracy: 0.2470
Epoch 5/10
63/63 [==============================] - 73s 1s/step - loss: -4.5900 - accuracy: 0.2720 - val_loss: -3.6446 - val_accuracy: 0.2470
Epoch 6/10
63/63 [==============================] - 73s 1s/step - loss: -4.5900 - accuracy: 0.2652 - val_loss: -3.6446 - val_accuracy: 0.2470
Epoch 7/10
63/63 [==============================] - 73s 1s/step - loss: -4.5900 - accuracy: 0.2627 - val_loss: -3.6446 - val_accuracy: 0.2470
Epoch 8/10
63/63 

In [48]:
### testing data

test_sentence = " I hate any one can hurt you "



predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['natural','Positive','Negative','irrervlent']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

1/1 [==============================] - 0s 47ms/step


'Positive'

### VADER

In [51]:
from nltk.sentiment import SentimentIntensityAnalyzer
#from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

### sentiment score

In [52]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [53]:
sia.polarity_scores('This is the worst thing ever.')

{'neg': 0.451, 'neu': 0.549, 'pos': 0.0, 'compound': -0.6249}

In [54]:
sia.polarity_scores('I hate the selfishness in you')

{'neg': 0.681, 'neu': 0.319, 'pos': 0.0, 'compound': -0.7506}

In [55]:
sia.polarity_scores('I hate any one can hurt you')

{'neg': 0.64, 'neu': 0.36, 'pos': 0.0, 'compound': -0.7964}

### Step 3. Roberta Pretrained Model

In [56]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [57]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [58]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [67]:
### FUNCTION THAT CALCULATE THE SENTIMENT SCORE

def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [59]:
example_1 = 'I hate the selfishness in you'
example_2 =  'I hate any one can hurt you'

In [68]:
print(polarity_scores_roberta(example_1))
print(polarity_scores_roberta(example_2))

{'roberta_neg': 0.9778666, 'roberta_neu': 0.01949323, 'roberta_pos': 0.0026402238}
{'roberta_neg': 0.9613866, 'roberta_neu': 0.035091653, 'roberta_pos': 0.0035216901}


### Transformer

In [63]:
from transformers import pipeline

sent_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [64]:
sent_pipeline('I love sentiment analysis!')

[{'label': 'POSITIVE', 'score': 0.9997853636741638}]

In [65]:
sent_pipeline('I hate the selfishness in you')

[{'label': 'NEGATIVE', 'score': 0.9951192140579224}]

In [66]:
sent_pipeline('I hate any one can hurt you')

[{'label': 'POSITIVE', 'score': 0.9785550236701965}]